In [0]:
%pip install openai
%pip install tiktoken
%pip install python-dotenv

Python interpreter will be restarted.
  Using cached openai-1.59.9-py3-none-any.whl (455 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached jiter-0.8.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (345 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached pydantic-2.10.5-py3-none-any.whl (431 kB)
  Using cached anyio-4.8.0-py3-none-any.whl (96 kB)
  Using cached exceptiongroup-1.2.2-py3-none-any.whl (16 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached pydantic_core-2.27.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.

In [0]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()
# Configura las credenciales directamente en el código
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openai-tajamar-1.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "Fvz3EQ5ZGgJKXwgnNDi8gPxZkfzKKjy3ZfMjNrtogXFYR5PIhF5JJQQJ99BAACYeBjFXJ3w3AAABACOGeKBA"

# Crear el cliente con las credenciales
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# A Few Shot Learning

In [0]:
# Zero-shot classification
system_prompt ="""Predict up to 5 emojis as a response to a text chat message. The output
should only include emojis.

input: The new visual design is blowing my mind 🤯
output: ➕,💘, ❤‍🔥

input: Well that looks great regardless
output: ❤️,🪄

input: Unfortunately this won't work
output: 💔,😔

input: sounds good, I'll look into that
output: 🙏,👍

input: 10hr cut of jeff goldblum laughing URL
output: 😂,💀,⚰️
"""
user_prompt = "The new user interface is amazing!"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":system_prompt},
                {"role":"user","content": user_prompt,}])
print(response.choices[0].message.content)

😍,👏,✨,💖,🎉


# Prompt Engineering Best Practices

## Write clear instructions

Examples:

-----------------------
Prompt:

Write code to calculate the Fibonacci sequence.

Better:

Write a TypeScript function to efficiently calculate the Fibonacci sequence. Comment the code liberally to explain what each piece does and why it's written that way.

----------------------

Prompt:

Summarize the meeting notes.

Better:

Summarize the meeting notes in a single paragraph. Then write a markdown list of the speakers and each of their key points. Finally, list the next steps or action items suggested by the speakers, if any.


## Role Playing

Examples:

-----------------------

System Message: When I ask for help to write something, you will reply with a document that contains at least one joke or playful comment in every paragraph.

----------------------


## Segment input text

Examples:

------------------------

user message: Summarize the text delimited by triple quotes with a haiku.

"""insert text here"""

------------------------

system message: You will be provided with a pair of articles (delimited with XML tags) about the same topic. First summarize the arguments of each article. Then indicate which of them makes a better argument and explain why.

user message: 

\<article> insert first article here \</article>

\<article> insert second article here \</article>

------------------------

## Explain steps and processes to complete a task

Examples:

--------------------------------

System Message:
Use the following step-by-step instructions to respond to user inputs.

Step 1 - The user will provide you with text in triple quotes. Summarize this text in one sentence with a prefix that says "Summary: ".

Step 2 - Translate the summary from Step 1 into Spanish, with a prefix that says "Translation: ".

---------------------------------

## Ejercicio 1: Clasificación de Emojis con Zero-shot
Enunciado:
Dado el siguiente texto, predice hasta 5 emojis que mejor representen la emoción o el tema del mensaje. Usa el modelo de Zero-shot para hacer la clasificación.

Texto: "Estoy tan feliz que no puedo dejar de sonreír."

Resultado esperado: 😄,😊,✨,🌞,❤️

In [0]:
user_prompt = "Estoy tan feliz que no puedo dejar de sonreír."
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"Eres un asistente que responde solo responde con emojis "},
                {"role":"user","content": user_prompt}],
        max_tokens=400,)
print(response.choices[0].message.content)




😄🎉😊✨


## Ejercicio 2: Traducción de Resumen
Enunciado:
El siguiente texto es una transcripción de una reunión. Resúmelo en una sola oración, luego traduce ese resumen al inglés.

Texto:
"""Hoy discutimos sobre la necesidad de mejorar la interfaz de usuario en el sitio web. Los participantes estuvieron de acuerdo en que se debe hacer más intuitiva y accesible, especialmente para usuarios mayores. También se mencionó la importancia de agregar soporte multilingüe."""



In [0]:
user_prompt = """Resúmelo en una sola oración, luego traduce ese resumen al inglés.### Hoy discutimos sobre la necesidad de mejorar la interfaz de usuario en el sitio web. Los participantes estuvieron de acuerdo en que se debe hacer más intuitiva y accesible, especialmente para usuarios mayores. También se mencionó la importancia de agregar soporte multilingüe."""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = [{"role":"system", "content":"Eres un asistente que resume el contenido en una sola oracion "},
                {"role":"user","content": user_prompt}],)
summary = response.choices[0].message.content
print(f"Summary: {summary}")

# Traducción al inglés
translation_prompt = f"Traduce el siguiente resumen al Ingles: {summary}"
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = [{"role":"system", "content":"Eres un traductor"},
    {"role":"user","content": translation_prompt}],)
print(f"Translation: {response.choices[0].message.content}")


Summary: Se discutió la necesidad de hacer la interfaz de usuario del sitio web más intuitiva y accesible, especialmente para usuarios mayores, y se consideró esencial agregar soporte multilingüe. 

We discussed the need to make the website's user interface more intuitive and accessible, especially for older users, and deemed it essential to add multilingual support.
Translation: We discussed the need to make the website's user interface more intuitive and accessible, especially for older users, and considered it essential to add multilingual support.


## Ejercicio 4: Respuesta Automática con Role-playing
Enunciado:
Configura un modelo para responder a una solicitud de escritura, haciendo que la respuesta contenga un toque de humor. Usa la siguiente solicitud como ejemplo.

Solicitud:
"Escribe una carta formal solicitando días de vacaciones."



In [0]:
system_prompt = "Eres un asistente humoristico todo lo que escribas tienen que tener un tono humoristico."

user_prompt = "Escribe una carta formal solicitando días de vacaciones."

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}]
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Ejemplo de salida: carta con broma


[Tu nombre]  
[Tu dirección]  
[Tu ciudad, provincia, código postal]  
[Tu correo electrónico]  
[Tu teléfono]  
[Fecha]  

[Nombre del jefe o supervisor]  
[Nombre de la empresa]  
[Dirección de la empresa]  

Estimado/a [Nombre del jefe o supervisor]:

Espero que esta carta lo/a encuentre en un excelente estado de humor, porque tengo una solicitud que podría contagiarle un poco de mi alegría. Si no está en la oficina, espero que al menos su planta de interior esté floreciendo maravillosamente.

Me gustaría solicitar unos días de vacaciones, específicamente desde el [fecha de inicio] hasta el [fecha de finalización]. He decidido que es hora de darle un merecido descanso a mi silla y, de paso, a mi cerebro, que me ha estado aleccionando sobre cuánto tiempo llevo sin salir de la oficina. Se siente un poco como un pez de circo atrapado en una pecera, y creo que hasta los peces de mis vacaciones me están dando miradas de desaprobación por no visitarlos.

Prometo que durante mi ausencia no

## Ejercicio 5: Segmentación de Texto
Enunciado:
Segmenta un texto delimitado por comillas triples y luego tradúcelo a otro idioma (por ejemplo, inglés). Aplica el siguiente formato de entrada de texto y muestra cómo lo harías.

Texto de Entrada:
"""Hoy es un buen día para aprender nuevas cosas y mejorar nuestras habilidades. ¡El futuro está lleno de oportunidades!"""


In [0]:
system_prompt = "Eres un traductor, traduces todo al ingles."

user_prompt = """Hoy es un buen día para aprender nuevas cosas y mejorar nuestras habilidades. ¡El futuro está lleno de oportunidades!"""

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}]
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Salida esperada: resumen en español e inglés


Today is a good day to learn new things and improve our skills. The future is full of opportunities!


## Ejercicio 6: Predicción de Emojis con Zero-shot
Enunciado:
Realiza una clasificación Zero-shot utilizando el modelo y predice hasta 5 emojis para el siguiente mensaje:

Texto de entrada:
"Estoy muy cansado, pero contento con el trabajo que he hecho."



In [0]:
system_prompt = """Predice hasta 5 emojis como respuesta a un mensaje de chat. La salida debe solo incluir emojis.
"""
user_prompt = "Estoy muy cansado, pero contento con el trabajo que he hecho."
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": system_prompt},
              {"role": "user", "content": user_prompt}]
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Ejemplo de salida: "😅,💪,👌"

💪😌✨😴👍


## Ejercicio 7: Predicción de Emojis con Few-shot Learning

Enunciado:

En este ejercicio utilizarás Few-shot learning con roles (system, user, assistant) para predecir emojis. En Few-shot learning, proporcionamos ejemplos específicos de interacciones para que el modelo aprenda a generalizar la tarea con pocos ejemplos.

Dado un conjunto de ejemplos de interacciones entre un usuario y un asistente, utiliza el modelo para predecir hasta 5 emojis para el siguiente mensaje:

Mensaje: "Estoy muy cansado, pero contento con el trabajo que he hecho."

Devuelve solo los emojis, separados por comas.

In [0]:
# Definir el prompt con Few-shot utilizando múltiples roles
few_shot_prompt = [
    {"role": "system", "content": "You are an assistant that predicts emojis based on user text."},
    
    {"role": "user", "content": "Estoy muy feliz con mi nuevo trabajo."},
    {"role": "assistant", "content": "😊,🎉,💼,💪"},
    
    {"role": "user", "content": "No me siento bien, tengo dolor de cabeza."},
    {"role": "assistant", "content": "😞,🤕,💔"},
    
    {"role": "user", "content": "Hoy es un día increíble para correr al aire libre."},
    {"role": "assistant", "content": "🏃‍♂️,🌳,🌞,💨"},
    
    {"role": "user", "content": "Acabo de terminar de ver una película emocionante."},
    {"role": "assistant", "content": "🎬,👏,😱,❤️"},
    
    {"role": "user", "content": "Voy a dormir temprano, me siento agotado."},
    {"role": "assistant", "content": "😴,🛏️,😴"},
    
    # Ahora pedimos la predicción para el nuevo mensaje
    {"role": "user", "content": "Estoy muy cansado, pero contento con el trabajo que he hecho."},
]

# Llamada a la API para obtener la predicción de emojis
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=few_shot_prompt
)

# Mostrar la respuesta generada
print(response.choices[0].message.content)  # Ejemplo de salida: "😅,💪,👌"

😌,💼,😅,👍
